In [1]:
import setup_django
setup_django.init()

In [2]:
import pandas as pd
from django.conf import settings

ratings_path = settings.DATA_DIR / "ratings_small.csv"
ratings_path.exists()

True

In [3]:
df = pd.read_csv(ratings_path)

In [4]:
df.head(n=10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [13]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [14]:
current_users = User.objects.all().values_list("id", flat=True)
rating_users = df["userId"].tolist()

In [15]:
missing_user_ids = set(rating_users) - set(current_users)
missing_user_ids

set()

In [16]:
for uid in missing_user_ids:
    User.objects.update_or_create(
        id = uid,
        username=f"missing-user-{uid}"
    )

In [17]:
import math
from decimal import Decimal

In [18]:
df["value"] = df["rating"].apply(lambda x: math.ceil(Decimal(x)))
df.head()

,userId,movieId,rating,timestamp,user_id,object_id,value
0,1,31,2.5,1260759144,1,31,3
1,1,1029,3.0,1260759179,1,1029,3
2,1,1061,3.0,1260759182,1,1061,3
3,1,1129,2.0,1260759185,1,1129,2
4,1,1172,4.0,1260759205,1,1172,4


In [19]:
df["user_id"] = df["userId"]
df["object_id"] = df["movieId"]

In [20]:
df.head()

,userId,movieId,rating,timestamp,user_id,object_id,value
0,1,31,2.5,1260759144,1,31,3
1,1,1029,3.0,1260759179,1,1029,3
2,1,1061,3.0,1260759182,1,1061,3
3,1,1129,2.0,1260759185,1,1129,2
4,1,1172,4.0,1260759205,1,1172,4


In [21]:
cols = ["user_id", "value", "object_id"]
transformed_df = df.copy()[cols]

In [23]:
rating_records = transformed_df.to_dict("records")

In [25]:
from ratings.models import Rating
qs = Rating.objects.all()
qs.delete()

(2043596, {'ratings.Rating': 2043596})

In [28]:
from django.contrib.contenttypes.models import ContentType

ctype = ContentType.objects.get(app_label="movies", model="movie")

In [31]:
new_ratings = []
for r in rating_records:
    r["content_type"] = ctype
    new_ratings.append(
        Rating(**r)
    )
Rating.objects.bulk_create(new_ratings, ignore_conflicts=True, batch_size=1000)


[<Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: Rating object (None)>,
 <Rating: 

In [32]:
from ratings.tasks import task_update_movie_ratings

task_update_movie_ratings()

Updated 9066 movie ratings in 0:00:05(5.085917949676514s)
